In [0]:
%run ./config

In [0]:
storage_account_name = os.getenv('STORAGE_ACCOUNT_NAME')
spark.conf.set(
    f'fs.azure.account.key.{storage_account_name}.blob.core.windows.net',
    os.getenv('STORAGE_ACCOUNT_KEY')
)

In [0]:
container_name = 'phishingurldetection'
csv_file_path = f'wasbs://{container_name}@{storage_account_name}.blob.core.windows.net/out.csv'

url_df = spark.read.format('csv').option('header', 'true').option('inferSchema','true').load(csv_file_path).createOrReplaceTempView('url_log')

In [0]:
spark.sql("""
        CREATE OR REPLACE TEMPORARY VIEW vw_url_agg AS
        SELECT label,
            COUNT(*) AS url_count,
            SUM(url_entropy) / COUNT(*) AS avg_url_entropy,
            SUM(CAST(dot_count AS DOUBLE)) / COUNT(*) AS avg_dot_count,
            SUM(CAST(at_count AS DOUBLE)) / COUNT(*) AS avg_at_count,
            SUM(CAST(dash_count AS DOUBLE)) / COUNT(*) AS avg_dash_count,
            100*SUM(CASE WHEN domain_has_digits THEN 1.0 ELSE 0.0 END) / COUNT(*) AS pct_with_digits
        FROM url_log
        WHERE label IN ('legitimate','phishing')
        GROUP BY label
""")

In [0]:
#display(spark.sql("""SELECT * FROM url_log WHERE label != 'legitimate' LIMIT 100"""))

In [0]:
import matplotlib.pyplot as plt

df = spark.sql("""SELECT * FROM vw_url_agg""")

pdf = df.toPandas()

plt.bar(pdf['label'], pdf['avg_url_entropy'])
plt.title('Average URL Entropy')
plt.show()